In [1]:
from azureml.core import Dataset
from azureml.core import Workspace
import datetime

ws = Workspace.from_config()
dataset = Dataset.get_by_name(ws, name='BTC-Bitstamp-raw')

df = dataset.to_pandas_dataframe()
df = df.rename(columns={"Timestamp": "dateid", "Close": "price","Volume_(BTC)":"volume"})


df

,dateid,Open,High,Low,price,volume,Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [2]:
dateparse = lambda x: datetime.datetime.utcfromtimestamp(float(x))
df['date'] = [dateparse(x) for x in df['dateid']]
df = df.set_index('date')
del df['dateid']
df['ask'] = df['price']*1.01
df['bid'] = df['price']*0.99
df = df.reindex(columns=['price','ask','bid','volume','High','Low'])
df

,price,ask,bid,volume,High,Low
date,,,,,,
2011-12-31 07:52:00,4.39,4.4339,4.3461,0.455581,4.39,4.39
2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,58714.31,58686.00
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,58693.43,58683.97
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,58723.84,58693.43


In [3]:
#df = df.dropna()

def addMa(timeframe):
    p=df['price']
    val =(p/p.rolling(timeframe).mean())-1
    df['ma'+timeframe] =val

def addMinMax(timeframe):
    p=df['price']
   
    mins = df['Low'].rolling(timeframe).min()
    df['maxmin'+timeframe]=(p - mins)/(df['High'].rolling(timeframe).max() - mins) 
    

def addBol(timeframe):
    p=df['price']
    val =(p/p.rolling(timeframe).mean())-1
    df['bol'+timeframe] = (val/p.rolling(timeframe).std()).clip(-3,3)

addMa('5min')
addMa('1h')
addMa('10D')
addBol('20D')
addMinMax('1D')


df = df.dropna()

    
del df['High']
del df['Low']

df

,price,ask,bid,volume,ma5min,ma1h,ma10D,bol20D,maxmin1D
date,,,,,,,,,
2011-12-31 16:59:00,4.57,4.6157,4.5243,37.862297,0.000000,0.000000,0.026966,0.259483,1.000000
2011-12-31 17:00:00,4.58,4.6258,4.5342,9.000000,0.001093,0.001093,0.021751,0.203496,1.000000
2012-01-01 04:16:00,4.58,4.6258,4.5342,1.502000,0.000000,0.000000,0.017326,0.169323,1.000000
2012-01-01 15:28:00,4.84,4.8884,4.7916,10.000000,0.000000,0.000000,0.061792,0.373183,1.000000
2012-01-01 22:45:00,5.00,5.0500,4.9500,10.100000,0.000000,0.000000,0.081917,0.363753,1.000000
...,...,...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,-0.000404,0.000310,0.056279,0.000017,0.704474
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,-0.000325,0.000313,0.056275,0.000017,0.704394
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,0.000236,0.000956,0.056959,0.000017,0.720389


In [4]:
df['bol15min'].max()

KeyError: 'bol15min'